<a href="https://colab.research.google.com/github/Srinivas1258/GCP/blob/master/Dance_form_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow

In [ ]:
!pip install pandas,numpy,matplotlib

In [ ]:
!pip install pandas

In [ ]:
!pip install numpy

In [ ]:
!pip install matplotlib

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
!unzip Dance_forms.zip

In [ ]:
ls

In [ ]:
!pip install keras-resnet

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,BatchNormalization,Dropout,Activation
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np


from tensorflow.keras.models import load_model
from keras.utils import np_utils
from keras import applications
from keras.preprocessing.image import ImageDataGenerator 
from keras import optimizers


In [ ]:
from tensorflow.keras.applications import ResNet50


In [ ]:
ls

In [ ]:
model1 = Sequential()
RESNET50_POOLING_AVERAGE = 'avg'

# 1st layer as the lumpsum weights from resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
# NOTE that this layer will be set below as NOT TRAINABLE, i.e., use it as is
model1.add(ResNet50(include_top = False, pooling = RESNET50_POOLING_AVERAGE, weights = 'imagenet'))

# 2nd layer as Dense for 2-class classification, i.e., dog or cat using SoftMax activation
model1.add(Dense(8, activation = 'softmax'))

# Say not to train first layer (ResNet) model as it is already trained
model1.layers[0].trainable = True

In [ ]:
model1.summary()

In [ ]:
model1.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


In [ ]:
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
IMAGE_RESIZE = 180
image_size = IMAGE_RESIZE

# preprocessing_function is applied on each image but only after re-sizing & augmentation (resize => augment => pre-process)
# Each of the keras.application.resnet* preprocess_input MOSTLY mean BATCH NORMALIZATION (applied on each batch) stabilize the inputs to nonlinear activation functions
# Batch Normalization helps in faster convergence
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

# flow_From_directory generates batches of augmented data (where augmentation can be color conversion, etc)
# Both train & valid folders must have NUM_CLASSES sub-folders
train_generator = data_generator.flow_from_directory(
        'train/',
        target_size=(image_size, image_size),
        batch_size=32,
        class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
        'valid/',
        target_size=(image_size, image_size),
        batch_size=32,
        class_mode='categorical') 

In [ ]:
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = 3)
cb_checkpointer = ModelCheckpoint(filepath = '../working/best.hdf5', monitor = 'val_loss', save_best_only = True, mode = 'auto')

In [ ]:
fit_history = model1.fit_generator(
        train_generator,
        
        epochs = 10,
        validation_data=validation_generator,
        
        callbacks=[cb_checkpointer, cb_early_stopper]
)
#model1.load_weights("../working/best.hdf5")

In [ ]:
acc=fit_history.history['accuracy']
val_acc=fit_history.history['val_accuracy']
loss=fit_history.history['loss']
val_loss=fit_history.history['val_loss']

epochs=range(len(acc))

fig = plt.figure(figsize=(20,10))
plt.plot(epochs, acc, 'r', label="Training Accuracy")
plt.plot(epochs, val_acc, 'b', label="Validation Accuracy")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc='lower right')
plt.show()
fig.savefig('../Accuracy_curve_CNN_256.jpg')

In [ ]:
test_generator = data_generator.flow_from_directory(
    directory = 'test/',
    target_size = (150, 150),
    batch_size = 32,
    class_mode = None,
    shuffle = False,
    seed = 123
)


In [ ]:
img=plt.imread('train/kathak/25.jpg')
plt.imshow(img)
plt.show()

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   rotation_range=15,
                                   vertical_flip=True,
                                   fill_mode='reflect',
                                   data_format='channels_last',
                                   brightness_range=[0.5, 1.5],
                                   featurewise_center=True,
                                   featurewise_std_normalization=True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
        'train/',
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        'valid/',
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

In [ ]:
model=Sequential()
model.add(Conv2D(filters=64,kernel_size=3,padding='same',input_shape=(150,150,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=2))

model.add(Conv2D(filters=64,kernel_size=3))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64,kernel_size=3))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=2))
#model.add(Dropout(0.25))

model.add(Conv2D(filters=64,kernel_size=3))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=2))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(BatchNormalization())
#model.add(Dropout(0.25))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Dense(8,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
''' model.fit(train_generator, epochs=10,
                              batch_size=32,
                              verbose=1,
                              validation_data=validation_generator
                            
                              ) '''

checkpoint = ModelCheckpoint('weights.hdf5', monitor='val_loss',
save_best_only=True)
# Store in a list to be used during training
#callbacks_list = [checkpoint]
# Fit the model on a training set, using the checkpoint as a
#callback
history = model.fit_generator(train_generator,
                              epochs=10,
                              
                              verbose=1,
                              validation_data=validation_generator
                            
                              )

In [ ]:
acc=history.history['acc']
val_acc=model.history['val_acc']
loss=model.history['loss']
val_loss=model.history['val_loss']

epochs=range(len(acc))

fig = plt.figure(figsize=(20,10))
plt.plot(epochs, acc, 'r', label="Training Accuracy")
plt.plot(epochs, val_acc, 'b', label="Validation Accuracy")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc='lower right')
plt.show()
fig.savefig('../Accuracy_curve_CNN_256.jpg')